# Neural Network

Importing the libraries


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import numpy as np

pd.set_option('display.max_columns', 200)

Creating a df with the disease rates for each city and their respective RISK value (1, if it belongs to a high risk cluster, and 0 otherwise).

In [2]:
cities = pd.read_csv('../../SatScan/muncod_risk_2015_to_2017.csv', index_col=[0])

disease = ""
path = '../../TabNet/Rates/'
all_files = glob.glob(path + "*.csv")
diseases_list = []
all_diseases = cities
for file in all_files:
    file_name = file.split("\\")[1]
    disease = file_name.split("_RATE")[0]
    disease_df = pd.read_csv(path + disease + '_RATE_15_17.csv', sep=',', index_col=0)
    disease_df = disease_df[["RATE", "MUNCOD"]]
    disease_df.rename(columns={"RATE": disease}, inplace=True)

    all_diseases = pd.merge(disease_df, all_diseases, left_on="MUNCOD", right_on="MUNCOD", how="outer")
df = all_diseases.drop('MUNCOD', 1)
df = df.drop('TRAUMATISMO_INTRACRANIANO', 1) # We remove Traumatismo Intracraniano (low p-value on Spearman test)
df = df.fillna(0)

Selecting X (independent variables) and y (dependent binary variable).

In [3]:
X = df.iloc[:, 0:14].values
y = df.iloc[:, -1].values

Splitting the dataset into the Training set and Test set

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

Feature Scaling

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Importing the Keras libraries and packages

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
C:\Users\pypas\anaconda3\envs\ML\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\pypas\anaconda3\envs\ML\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\pypas\anaconda3\envs\ML\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\pypas\anaconda3\envs\ML\lib\site-packages\tenso

Initialising the ANN

In [8]:
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 14))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 50, epochs = 100)

Epoch 1/100
4243/4243 [==============================] - 0s 90us/step - loss: 0.6910 - accuracy: 0.5454
Epoch 2/100
4243/4243 [==============================] - 0s 34us/step - loss: 0.6609 - accuracy: 0.6927
Epoch 3/100
4243/4243 [==============================] - 0s 35us/step - loss: 0.5869 - accuracy: 0.7294
Epoch 4/100
4243/4243 [==============================] - 0s 34us/step - loss: 0.5456 - accuracy: 0.7351
Epoch 5/100
4243/4243 [==============================] - 0s 35us/step - loss: 0.5345 - accuracy: 0.7370
Epoch 6/100
4243/4243 [==============================] - 0s 34us/step - loss: 0.5300 - accuracy: 0.7407
Epoch 7/100
4243/4243 [==============================] - 0s 35us/step - loss: 0.5273 - accuracy: 0.7462
Epoch 8/100
4243/4243 [==============================] - 0s 36us/step - loss: 0.5257 - accuracy: 0.7473
Epoch 9/100
4243/4243 [==============================] - 0s 34us/step - loss: 0.5247 - accuracy: 0.7483
Epoch 10/100
4243/4243 [==============================] - 0s 34u

4243/4243 [==============================] - 0s 39us/step - loss: 0.5037 - accuracy: 0.7587
Epoch 79/100
4243/4243 [==============================] - 0s 39us/step - loss: 0.5030 - accuracy: 0.7589
Epoch 80/100
4243/4243 [==============================] - 0s 35us/step - loss: 0.5032 - accuracy: 0.7589
Epoch 81/100
4243/4243 [==============================] - 0s 36us/step - loss: 0.5032 - accuracy: 0.7608
Epoch 82/100
4243/4243 [==============================] - 0s 39us/step - loss: 0.5029 - accuracy: 0.7589
Epoch 83/100
4243/4243 [==============================] - 0s 38us/step - loss: 0.5026 - accuracy: 0.7617
Epoch 84/100
4243/4243 [==============================] - 0s 38us/step - loss: 0.5026 - accuracy: 0.7627
Epoch 85/100
4243/4243 [==============================] - 0s 33us/step - loss: 0.5019 - accuracy: 0.7596
Epoch 86/100
4243/4243 [==============================] - 0s 34us/step - loss: 0.5019 - accuracy: 0.7601
Epoch 87/100
4243/4243 [==============================] - 0s 39us/st

Predicting the Test set results

In [9]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

Making the Confusion Matrix

In [10]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[652, 103],
       [222, 438]], dtype=int64)

In [11]:
from sklearn import metrics
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Precision: 0.8096118299445472
Recall: 0.6636363636363637
